# ✍️ 02_rag_query_generate_script.ipynb
Generate a YouTube script from the latest cleaned Reddit posts using FAISS + GPT-4

In [ ]:
%pip install faiss-cpu openai python-dotenv tiktoken

In [ ]:
import os
import json
import openai
import faiss
import numpy as np
from dotenv import load_dotenv
from sklearn.preprocessing import normalize
from datetime import datetime

In [ ]:
# ✅ Load environment
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# ✅ Load inputs
with open("../inputs/idea.json", encoding="utf-8") as f:
    idea = json.load(f)
with open("../inputs/profile.json", encoding="utf-8") as f:
    profile = json.load(f)
with open("../inputs/template_youtube_script.json", encoding="utf-8") as f:
    template = json.load(f)

In [ ]:
# ✅ Load FAISS index + metadata
index = faiss.read_index("../data/vectorstore/reddit_index.faiss")
with open("../data/vectorstore/reddit_metadata.json", encoding="utf-8") as f:
    metadata = json.load(f)

In [ ]:
# ✅ Embed idea topic
def get_embedding(text):
    return openai.Embedding.create(
        input=[text.replace("\n", " ")],
        model="text-embedding-ada-002"
    )['data'][0]['embedding']

query_embedding = get_embedding(idea["topic"])
query_embedding = normalize(np.array([query_embedding]).astype("float32"))

In [ ]:
# ✅ Search FAISS index
D, I = index.search(query_embedding, 5)
top_context = [metadata[i] for i in I[0]]

In [ ]:
# ✅ Build prompt
references = "\n\n".join([f"[Source]({m['url']}) - {m['created']}" for m in top_context])

prompt = f"""
You are a scriptwriter creating a YouTube video using the style and insights of Steve Jobs.
Topic: {idea['topic']}
Profile: {profile}
Template: {template}

Here are references to support your script:
{references}

Generate a compelling, inspiring, educational, and persuasive script in 2-3 minutes of speaking.
"""

In [ ]:
# ✅ GPT-4 completion
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a visionary game industry scriptwriter."},
        {"role": "user", "content": prompt}
    ]
)
script = response.choices[0].message.content.strip()

In [ ]:
# ✅ Save to file
output_dir = "../outputs/youtube_scripts"
os.makedirs(output_dir, exist_ok=True)
filename = f"script_{datetime.now().strftime('%Y-%m-%d')}.txt"
with open(os.path.join(output_dir, filename), "w", encoding="utf-8") as f:
    f.write(script)
print(f"✅ Script saved to: {filename}")